# 1.0  Data Preparation

## GADM Shapefile (Admin Boundaries)

We use the GADM shapefile to load Kenya's county and sub-county boundaries. This helps us link food price and weather data to specific regions for analysis and visualization.


In [1]:
!wget -q https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_KEN_2.json.zip && unzip -oq gadm41_KEN_2.json.zip && rm gadm41_KEN_2.json.zip

In [2]:
#importing libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

## 1.1 Load Datasets

We load food prices, market information, rainfall data, and Kenya's administrative boundaries to begin the data preparation process.


In [ ]:
# Load food price dataset
prices = pd.read_csv("wfp_food_prices_ken.csv", comment='#')

# Load market locations and metadata
markets = pd.read_csv("wfp_markets_ken.csv", comment='#')

# Load county and sub-county boundaries (GADM Level 2)
counties = gpd.read_file("gadm41_KEN_2.json")

# Load county-level monthly rainfall data
rainfall = pd.read_csv("kenya_county_rainfall.csv")
